# Webscrapping de datos del sitio de compras de la Municipalidad de General Pueyrredón


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
# Definimos la URL para scrappear
url = 'https://appsvr.mardelplata.gob.ar/Consultas07/OrdenesDeCompra/OC/index.asp?fmANIO_CON=2022&fmJURISDICCION_CON=1110101000&fmTIPOCONT_CON=--&fmNRO_OC=&Consultar=Consultar' 

# Hacemos el request de la url, y luego lo metemos en BeautifulSoup
html = requests.get(url.strip(), headers={'User-Agent': 'Mozilla/5.0'}).text
soup = BeautifulSoup(html, "lxml")


In [193]:
# Generamos una lista vacía, que la vamos a utilizar para cargar los elementos scrappeados de la tabla
data = []

# Iteramos a través de todas las filas (tr) de tabla. Cada fila por la que navegamos, tiene columnas,
# por la que también tenemos que navegar para extraer los datos.
# Por lo que hacemos dos iteraciones, una por las filas, y en cada fila, por cada columna. Y ahi obtenemos
# el dato para agregar a la base

# Iteramos por las filas de la tabla (tr)
for row in soup.find_all('tr'):
    # De cada fila, obtenemos las columnas (td) con un find_all dentro de la fila row. Pero de las columnas, solo
    # buscamos las que tienen la clase "textocomun"
    columns = row.find_all('td', class_='textocomun')
    
    # Definimos una lista td para cargar toda la info que obtenemos de cada columna
    td = []
    
    # Iteramos por cada columna columns
    for cols in columns:
        # Eliminamos los espacios del comienzo y final de la cadena
        cols = cols.text.strip()
        # Agregamos cols a un lista que se llama td. Cuando termina el bucle cols, td es una lista 
        # donde cada elemento es el valor de una columna de UNA fila
        td.append(cols)
        
    # Fuera del bucle cols, tenemos td con la info de la fila. Tenemos que agregarlo a data, que es otra lista
    # donde cada elemento de la lista es una fila, con cada elementode esa segunda lista siendo el valor de cada
    # columna.
    # Si td no está vacío (sería una row vacía) o si no tiene 'Formulario de consulta' en la posición 0, 
    # agregamos a td a data
    if (len(td) != 0) and (td[0]!='Formulario de Consulta'):
        data.append(td) 

In [194]:
# En la variable columna definimos cuales serían los nombres de las columnas del dataframe.
# La primer columna se llama remove, porque la vamos a borrar porque es una columna vacía
columnas = ['remove', 'oc','fecha_emision','importe','proveedor','tipo_contratacion','observaciones','estado']
df = pd.DataFrame(data, columns = columnas)

In [195]:
# Eliminamos la columna remove
df = df[['oc', 'fecha_emision', 'importe', 'proveedor',
       'tipo_contratacion', 'observaciones', 'estado']]

In [196]:
df

,oc,fecha_emision,importe,proveedor,tipo_contratacion,observaciones,estado
0,1000,16/8/2022,"$338.400,00",BERTOLAMI MABEL ELENA,Compra Directa Varios Proveedores,Destino: Item 1: 8 matafuegos para el Archivo ...,Registrada
1,564,17/5/2022,"$22.000,00",AGUIRRE RICARDO OSVALDO,Compra Directa Varios Proveedores,LIBRE DE TODO GASTO ADICIONAL.,Registrada
2,147,18/2/2022,"$273.000,00",CAZOU RICARDO MARTIN,Concurso de precios,Expte 7895-6-2021 cpo 1 alc 2 cpo 1. Prorroga ...,Registrada
3,155,21/2/2022,"$13.000,00",ARIAS JUAN ANIBAL,Compra Directa Varios Proveedores,"carga, descarga y flete a cargo del proveedor....",Registrada
4,951,8/8/2022,"$2.880.000,00",CAZOU RICARDO MARTIN,Licitación Privada,Expte 3623 Dig. 8 Año 2022 Cpo. 1. Licitacion ...,Registrada
5,932,3/8/2022,"$26.273,40",RADIOGRAFICA OESTE SRL,Licitación Pública,Expediente Nº 2731-5-2022 Cpo.6 Licitación Púb...,Registrada
6,506,6/5/2022,"$75.000,00",FEUER DIEGO MANUEL,Compra Directa Varios Proveedores,PERIODO: DESDE LA NOTIFICACIÓN DE LA ORDEN DE ...,Registrada
7,45,25/1/2022,"$85.550,00",CURRA ANTONIO,Compra Directa Varios Proveedores,"CARGA, DESCARGA Y FLETE A CARGO DEL PROVEEDOR....",Registrada
8,427,22/4/2022,"$38.400,00",SUAREZ CENTURION GABRIEL ALEJANDRO MATIAS,Compra Directa Varios Proveedores,"CARGA, DESCARGA Y FLETE A CARGO DEL PROVEEDOR....",Registrada
9,871,15/7/2022,"$196.528,50",MALTHUS SA,Compra Directa Varios Proveedores,LUGAR DE ENTREGA: SEC CULTURA OLAVARRIA 2508 1...,Registrada


## Generalizamos el script para todas las dependencias

In [3]:
jurisdicciones = [1110101000]
#                   1110111000]


In [4]:
def init_bs(jur):
    # Definimos la URL para scrappear, armándola con el codigo de jurisdicción
    url = 'https://appsvr.mardelplata.gob.ar/Consultas07/OrdenesDeCompra/OC/index.asp?fmANIO_CON=2022&fmJURISDICCION_CON=' + str(jur)+'&fmTIPOCONT_CON=--&fmNRO_OC=&Consultar=Consultar' 
    print(url)
    # Hacemos el request de la url, y luego lo metemos en BeautifulSoup
    html = requests.get(url.strip(), headers={'User-Agent': 'Mozilla/5.0'}).text
    soup = BeautifulSoup(html, "lxml")
    print('conectado a ' + str(jur))
    return soup

In [13]:
def scrapper(soup, jur, columnas):
    print(soup)
    # Generamos una lista vacía, que la vamos a utilizar para cargar los elementos scrappeados de la tabla
    data = []
    # Iteramos a través de todas las filas (tr) de tabla. Cada fila por la que navegamos, tiene columnas,
    # por la que también tenemos que navegar para extraer los datos.
    # Por lo que hacemos dos iteraciones, una por las filas, y en cada fila, por cada columna. Y ahi obtenemos
    # el dato para agregar a la base
    # Iteramos por las filas de la tabla (tr)
    tr = soup.find_all('tr')
    for row in tr:
        print(row)
        # De cada fila, obtenemos las columnas (td) con un find_all dentro de la fila row. Pero de las columnas, solo
        # buscamos las que tienen la clase "textocomun"
        columns = row.find_all('td', class_='textocomun')
        print(columns)
        td = []

        # Iteramos por cada columna columns
        for cols in columns:
            print(cols)
            # Eliminamos los espacios del comienzo y final de la cadena
            cols = cols.text.strip()
            print(cols)
            # Agregamos cols a un lista que se llama td. Cuando termina el bucle cols, td es una lista 
            # donde cada elemento es el valor de una columna de UNA fila
            td.append(cols)
            td.append(jur)
        # Fuera del bucle cols, tenemos td con la info de la fila. Tenemos que agregarlo a data, que es otra lista
        # donde cada elemento de la lista es una fila, con cada elementode esa segunda lista siendo el valor de cada
        # columna.
        # Si td no está vacío (sería una row vacía) o si no tiene 'Formulario de consulta' en la posición 0, 
        # agregamos a td a data
#         if (td[0]!='Formulario de Consulta'):
        data.append(td)
#         print(td)
#             print(data)
        print('scrappeado ' + str(jur))
#         df = pd.DataFrame(data, columns = columnas)
#         print(df)
        return data

In [15]:
columnas = ['remove', 'oc','fecha_emision','importe','proveedor','tipo_contratacion','observaciones','estado']
df = pd.DataFrame()
for jur in jurisdicciones:
    soup = init_bs(jur)
#     print(soup)
#     print('===========================================================')
#     print('===========================================================')
#     print('===========================================================')
#     print('===========================================================')
#     print('===========================================================')
#     print('===========================================================')
#     print('===========================================================')
    
    data = scrapper(soup,jur, columnas)
#     print(data)
#     print('===========================================================')
#     print('===========================================================')
#     print('===========================================================')
    
    time.sleep(10)

https://appsvr.mardelplata.gob.ar/Consultas07/OrdenesDeCompra/OC/index.asp?fmANIO_CON=2022&fmJURISDICCION_CON=1110101000&fmTIPOCONT_CON=--&fmNRO_OC=&Consultar=Consultar
conectado a 1110101000
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Drupal 7 (http://drupal.org)" name="Generator"/>
<script src="/cdn-cgi/apps/head/sePLM1HXS1RiY2TtVBYEd0IAO8I.js"></script><link href="https://www.mardelplata.gob.ar/sites/all/themes/mgp/ico/xfavicon.ico.pagespeed.ic.Xz6twAEkcD.png" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<title>Error 500 | Sitio Oficial del Municipio de General Pueyrredon</title>
<link href="https://www.mardelplata.gob.ar/A.modules,,_system,,_system.base.css,,qrgfz68+misc,,_ui,,_jquery.ui.core.css,,qrgfz68+misc,,_ui,,_jquery.ui.theme.css,,qrgfz68+misc,,_ui,,_jquery.ui.datepicker.css,,qrgfz68+sites,,_all

KeyboardInterrupt: 

In [12]:
data

In [40]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head>
<title>Datos</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script src="/cdn-cgi/apps/head/sePLM1HXS1RiY2TtVBYEd0IAO8I.js"></script><link href="/css/A.style.css.pagespeed.cf.JXP4QKHULD.css" rel="stylesheet" type="text/css"/>
<style>#titulo{position:relative;left:50 px;border 1px #CCCCCC}#subtitulo{position:relative;left:50 px;border 1px #CCCCCC}#descripcion{position:relative;left:50 px;border 1px #CCCCCC}#busqueda{position:relative;left:50 px;border 1px #CCCCCC;width:503px;height:189px}#listado_solicitudes{position:relative;left:50 px;border 1px #CCCCCC}#detalle_solicitud{position:relative;left:50 px;border 1px #CCCCCC}.input_text{font-family:Verdana,Arial,Helvetica,sans-serif;font-size:12px;color:#000;border:1px solid #ccc}.input_class{font-family:Verdana,Arial,Helvetica,sans-serif;font-size:10px;font-weight:bold;color:#fff;background-color:#666;margin:1px;padding-top:2px;borde

In [48]:
row = soup.find_all('tr')
for col in row.find('td'):
    print(col)

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?